### GreenDS

# Fundamentals of Agro-Environmental Data Science

## Example APIs and Web scraping

### Introduction

The purpose of this Jupyter Notebook exercise is to demonstrate the methods available to obtain data from online services. Two examples are explored:
- web data services based on REST APIs
- web scraping from online web pages

Sometime, web pages use APIs to expose information and services, but no documentation is provided. We will learn how to identify the existence of these services, to use them in more efficient data collection.

## Web scraping Air Quality data

The QUALAR (https://qualar.apambiente.pt/) is a web platform of APA, the Portuguese Environment Agency, that displays online air quality data sampled by on air monitoring stations in Portugal. Unfortunately, the platform does not expose to the final user, or provides documentation how to use the API service that was implemented for web users downloads. Downloads are generated as XLSX files.

However, it is possible to hack the source code of the webpage to identify that there is an implemented API, and that it can be used to facilite efficient download of data. This exercise will demonstrate that, with the following steps:
- check how are downloads generated from the website
- identify and collect the parameters that define the data download
- use the API to download data
- as a bonus, visualise a timeseries of a data quality variable.

## Prepare your environment

1. Create two directories inside your projects' directory named `raw-data` and `qualar-data`, with the following command:

In [ ]:
!mkdir raw-data qualar-data

## 1. Data download for human users

Visit the data download page of QUALAR, at https://qualar.apambiente.pt/downloads. It displays a table with a list of Air Monitoring Stations, with the following columns:
- Region (Região)
- Municipality (Concelho)
- Station (Estação)
- Station type (Tipo de Estação), with categories traffic, industrial and background
- Área type (Tipo de Área), with categories urban, suburban and rural
- columns for the following pollutants: O3, NO2, CO, SO2, PM10, PM2.5, C6H6, other

On the top, the page has two fields to define the time range for the data download, and on the left several buttons to activate filters about the type of station and type of área.

To make a download, users can click on arrows that are available for each station and each pollutant, or if they want to download all pollutants for a station, they can click directly on the station name. After clicking, the download file is generated for the requested options as a excel file (xlsx).

*Try to make a download in this way, and check the file downloaded.*

## 2. Verify how are downloads generated by inspecting the webpage source code

It is possible to inspect the source code of the table, and the behaviour of the page when a download is solicited. Checking this we can try to identify which methods are used to provide data to users. If we manage to verify that the web page is served by an API, and we can identify which parameters define a request, then it would be possible to generate a script to speed up downloads.

**1. Activate the Inspect Tool of the source code of the webpage.**

*Open in your web browser, navigate to https://qualar.apambiente.pt/downloads.Afterwards, in the menu of your browser, find the option **Web Developer Tools** or **Developer Tools** (in Firefox or Chrome, you will find it in **More tools**). This will open a new panel in the browser.*

**2. Check the method to generate downloads**

As mentioned before, clicking on the name of a station will generate a download with all data for that station. This means that through the HTTP protocol, a request is made through the network. Checking which request was made (which URL request was send) is a good way of verifying what was the information send to the web server.

*On the Developer Panel, click on the tab **Netwotk**. After that, click on the name of a station to make a download request. This will generate a new row on the panel, with the information about a **GET** request.*

One of the parameters in that row is the name or file field, which shows the URL sent to the server, e.g.:

```https://qualar.apambiente.pt/api/download.php?poluente_id=0&estacao_id=3082&data_inicio=2021-01-01&data_fim=2021-12-31&influencias=1,2,3&ambientes=1,2,4```

We can identify the following sections in the URL:

Host URL: ```https://qualar.apambiente.pt/api/download.php```

Parameters:```poluente_id=0&estacao_id=3082&data_inicio=2021-01-01&data_fim=2021-12-31&influencias=1,2,3&ambientes=1,2,4```

The meaning of the parameters is more or less obvious:

`poluente_id` - the ID of the pollutant. The value zero should mean all pollutants

`estacao_ID` - the ID of the station

`data_inicio` - starting date

`data_fim` - ending date

`influencias` - station type

`ambientes` - area type

**3. Verify that the method for download works**

We have just identified an API service for downloading data. We can check if it works, testing with different parameters and see if results correspond to what is expected:

*To download data only for **year 2020**, try the following modified URL:*

```https://qualar.apambiente.pt/api/download.php?poluente_id=0&estacao_id=3082&data_inicio=2020-01-01&data_fim=2020-12-31&influencias=1,2,3&ambientes=1,2,4```

The challenge now is to discover the values of the IDs of the air monitoring stations (the parameter `estacao_ID`). If we find these, we can make a script to make automatic requests to download the data files. 

**4. Inspect the HTML source code** 

On the top left bar of the **Developer tools** panel, there is a arrow cursor option. Select this, and the place the mouse pointed on the name of one air monitoring station, in the table. You will verify that for each section of the web page where you hover your mouse, the corresponding HTML source code will be highlighted in the developer panel.

**5. Select the section of the HTML code with the cell of the station name**

Place the mouse so that the complete cell with the name of a station in the table is highlighted, and the click. In the source code, a line starting with the tag **td** should be selected.

In the begining of that line, a triangle indicates that the inner HTLM code can be expanded. Remember that HTML is a hierarchical language, where html tags placed inside other tags are "child" or "depended" of these.

The html line looks, for example, like the following:

```html
<td style="background-color: #EBF7FF; text-align: center; vertical-align: middle; ">
    <label title="Dados de todos os poluentes para uma estação num dado ano" 
           style="color:#0000ff; cursor:pointer" onclick="tableDataManager.openExcel(3082)">
        <u><b>Alfragide/Amadora</b></u>
    </label>
</td>
```

The interesting about that html is that the **onclick** event on the **label** tag is an actionable event that triggers a method to open an Excel, **with an ID 3082**. This was for the station Alfragide/Amadora. If we try another station, the ID will be different. We have, therefore, found a way of identifying the IDs of all air monitoring stations.

## 3. Scrap the HTML source code to obtain IDs of the stations

We will scrap the html of table in https://qualar.apambiente.pt/downloads to obtain the IDs of the air monitoring stations. For that, we will use the phyton module **BeautifulSoup**.

**1. Get the table content**

In the web page, the table content is loaded dynamically. It is possible to use python libraries to obtain that html code, but it is easier to do it manually.
- go to the Inspector or Elements panel of the Developer Tool in your browser, depending of which browser you use
- navigate down through the html code hierarchy until you find the html tag `<tbody>`. Verify in the web page that the correct section of the table containing data is highlighted.
- right-click on top of the `<tbody>` tag, and select Copy --> Copy Outer HTML
- paste the memory context to a new file in your text editor (VS Code, Notepad++, etc), and save it with the name *qualar_table.html* in the `raw-data` o your project.

**2. Install and import** *BeautifulSoup*

In [ ]:
# If you don't have BeautifulSoup library installed, you can do it at the shell terminal
# with the following commands:
#
# $ pip3 install BeautifulSoup4

In [ ]:
# import BeautifulSoup and open the html file
from bs4 import BeautifulSoup

**3. Get air pollution station IDs**

Extract the station ID from the html text using BeautifulSoup. We will store the list of station IDs in a list to use latter.

In [ ]:
# open the html file
with open("./raw-data/qualar_table.html") as html_table:
    soup = BeautifulSoup(html_table, 'html.parser')

# create a list to store ids
station_ids = []


# get ids from html
for row in soup.find_all('label'):
    row_text = row['onclick']
    ids = row_text[row_text.find( '(' )+1:row_text.find( ')' )]
    station_ids.append(ids)
    
print(station_ids)

## 4. Download files using the API

We can now make requests to the API to download xlsx files. First, let's import the python modules.

In [ ]:
# import necessary libraries
import requests
import datetime
import pandas as pd
from time import sleep
import urllib

After that, we will request to the REST API the excel files for each stattion, identified by its ID, and store them localy. 

In [ ]:
# Define base URL of the service
base_url = 'https://qualar.apambiente.pt/api/download.php'

# Define params for the API request
params = {"poluente_id":"0", "influencias":"1,2,3", "ambientes":"1,2,4"}

# Define begining of the data period. You can change this for another year
params["data_inicio"] = "2021-01-01"

# Define end of the data period. You can change this for another year
params["data_fim"] = "2021-12-31"

# As this is not in production, we will replace the full list of ids by a smaller list of ids, to save time. 
# If you want all data, comment the next line
station_ids = ['3095', '3103', '3104', '3096', '3070']

# This iteration will go through the list of IDs, request the excel file based on the URL and save it locally.
for id_value in station_ids:
    # define name of file to be saved
    filename = './qualar-data/station_'+ id_value + '.xlsx'
    # add station_id to the parameters of the URL
    params["estacao_id"] = id_value
    try:
        r = requests.get(base_url, params)
    except requests.exceptions.RequestException as e: 
        raise SystemExit(e)
    print(r.url)
    with open(filename, 'wb') as f:
        f.write(r.content)

## 5. Preview dowloaded data

We can import xlsx data to Pandas, and make a plot.

In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt

Let's print the list of available excel files:

In [ ]:
# list xlsx files in the data directory

files = os.listdir('./qualar-data')
print(files)

The following imports the files to a Pandas dataframe, and creates a plot with all variables for each station.

In [ ]:
# For each file, import to a pandas dataframe and make a plot
for file in files:
    # define file name
    xls = './qualar-data/'+file
    # read to a pandas dataframe
    df = pd.read_excel(xls)
    
    # create a list with column names of the dataframe 
    col_names = list(df.columns)
    
    # Convert the column with dates to a datetime format
    df['Dates'] = pd.to_datetime(df[col_names[0]])

    # plot all variables, setting dates as the index
    df.set_index('Dates').plot()